In [191]:
from pathlib import Path
from pprint import pprint
from collections import defaultdict
import re
import pandas as pd
import re
import csv

%reload_ext autoreload
%autoreload 2

In [127]:
def extract(file_contents, regex, post=None):
    # Search for the pattern in the text
    match = re.search(regex, file_contents)

    if match:
        # Extract first capture group
        if post:
            temp = post(match.group(1))
        else:
            temp = match.group(1)
        return temp
    else:
        return ""

In [128]:
NUMBER_OF_BIDDERS = "NUMBER OF BIDDERS"
BID_OPENING_DATE = "BID OPENING DATE"
CONTRACT_NUMBER = "CONTRACT NUMBER"
TOTAL_NUMBER_OF_WORKING_DAYS = "TOTAL NUMBER OF WORKING DAYS"
CONTRACT_ITEMS = "CONTACT ITEMS"
TYPE = "TYPE"
PERCENT_OVER_EST = "PERCENT OVER EST"
PERCENT_UNDER_EST = "PERCENT UNDER EST"
ENGINEERS_EST = "ENGINEERS EST"
AMOUNT_OVER = "AMOUNT OVER"
CONTRACT_CODE = "CONTRACT CODE"
AMOUNT_UNDER = "AMOUNT UNDER"

BID_RANK = "BID RANK"
BID_TOTAL = "BID TOTAL"   
BIDDER_ID = "BIDDER ID"
BIDDER_INFORMATION = "BIDDER INFORMATION  (NAME/ADDRESS/LOCATION)"

In [174]:
l = []

for filepath in Path('/Users/nenadbozinovic/Documents/ocr/RR Procurement - Raw Data/Txt files - lineprinter').glob('*.txt'):
    
    # Open the file in read mode ('r')
    with open(filepath, 'r') as file:
        # Read the contents of the file into a string
        file_contents = file.read()
    row = defaultdict(str)
    # row['filepath'] = filepath
    row['filename'] = filepath.stem
    row[BID_OPENING_DATE] = extract(file_contents, r"BID OPENING DATE\s+(\d+/\d+/\d+)")
    row[CONTRACT_NUMBER] = extract(file_contents, r"CONTRACT NUMBER\s+([A-Za-z0-9-]+)")
    row[CONTRACT_CODE] = extract(file_contents, r"CONTRACT CODE\s+'([^']+)'")
    row[CONTRACT_ITEMS] = extract(file_contents, r"(\d+)\s+CONTRACT ITEMS")
    row[TOTAL_NUMBER_OF_WORKING_DAYS] = extract(file_contents, r"TOTAL NUMBER OF WORKING DAYS\s+(\d+)")
    row[NUMBER_OF_BIDDERS] = extract(file_contents, r"NUMBER OF BIDDERS\s+(\d+)")
    row[ENGINEERS_EST] = extract(file_contents, r"ENGINEERS EST\s+([\d,]+\.\d{2})")
    row[AMOUNT_OVER] = extract(file_contents, r"AMOUNT OVER\s+([\d,]+\.\d{2})")
    row[AMOUNT_UNDER] = extract(file_contents, r"AMOUNT UNDER\s+([\d,]+\.\d{2})")
    row[PERCENT_OVER_EST] = extract(file_contents, r"PERCENT OVER EST\s+(\d+)")
    row[PERCENT_UNDER_EST] = extract(file_contents, r"PERCENT UNDER EST\s+(\d+)")
    row[TYPE] = extract(file_contents, r"(.*?)\s+FEDERAL AID", post=lambda x: x.lstrip())
    
    

    l.append(row)
a = pd.DataFrame(l)
a


,filename,BID OPENING DATE,CONTRACT NUMBER,CONTRACT CODE,CONTACT ITEMS,TOTAL NUMBER OF WORKING DAYS,NUMBER OF BIDDERS,ENGINEERS EST,AMOUNT OVER,AMOUNT UNDER,PERCENT OVER EST,PERCENT UNDER EST,TYPE
0,01-0A1004.pdf_7718,03/22/16,01-0A1004,A,42,100,9,"1,467,088.00",,"395,859.50",,26,BRIDGE SEISMIC RETROFIT
1,01-0A0704.pdf_1785,05/19/10,01-0A0704,B,17,40,5,"4,440,262.00",,"410,525.00",,9,PLACE HOT MIX ASPHALT
2,01-0A0204.pdf_2638,03/15/11,01-0A0204,E,14,25,2,"1,022,104.00","15,775.00",,1,,RUBBERIZED HMA OVERLAY
3,01-0A1204.pdf_11468,05/27/20,01-0A1204,D,78,,,,,,,,BRIDGE SEISMIC RETROFIT
4,01-0A1504.pdf_4994,04/23/13,01-0A1504,D,31,30,2,"1,823,447.50","155,717.50",,8,,RUBBERIZED HOT MIX ASPHALT
5,01-0A0804.pdf_2923,04/27/11,01-0A0804,E,17,25,3,"838,032.50","42,776.50",,5,,BONDED WEARING COURSE
6,01-0A0404.pdf_10165,09/27/18,01-0A0404,A,115,130,7,"10,366,370.00",,"2,718,364.50",,26,SHOULDER WIDENING
7,01-0A1204.pdf_11495,06/04/20,01-0A1204,E,78,130,3,"2,544,443.50",,"346,874.00",,13,BRIDGE SEISMIC RETROFIT
8,01-0A0904.pdf_2724,03/29/11,01-0A0904,H,15,20,6,"356,785.00","62,635.00",,17,,SEAL COAT
9,01-0A2004.pdf_4038,05/08/12,01-0A2004,I,14,20,6,"908,382.00","82,290.00",,9,,REPLACE AC SURFACING


In [117]:
a.to_csv('output.csv', index=False)

In [194]:
l = []

for filepath in Path('/Users/nenadbozinovic/Documents/ocr/RR Procurement - Raw Data/Txt files - lineprinter').glob('*.txt'):
    
    # Open the file in read mode ('r')
    with open(filepath, 'r') as file:
        # Read the contents of the file into a string
        file_contents = file.read()
    row = defaultdict(str)
    # row['filepath'] = filepath
    row['filename'] = filepath.stem
    
    # Regular expression to match each entry
    pattern = re.compile(r"(\d+)\s+(A\))?\s+([\d,]+\.\d{2})\s+(\d+)\s+(.*?)\s+(\d{3} \d{3}-\d{4})(.*?)\s+(\d+)\s+(B\))?")

    # Find all matches
    matches = pattern.findall(file_contents)

    # Prepare CSV data
    csv_data = [["BID RANK", "OPTIONAL_A", "BID TOTAL", "BIDDER ID", "BIDDER NAME", "BIDDER PHONE", "OPTIONAL TEXT", "BIDDER ID", "OPTIONAL_B"]]
    for match in matches:
        # Combine the captured groups into a single row, note that some post-processing may be needed depending on the exact format you want
        row = [match[0], match[1], match[2], match[3].strip(), match[4], match[5].strip(), match[6], match[7].strip(), match[8]]
        csv_data.append(row)
        
    # Write CSV data to file
    with open(f"bids_{filepath.stem}.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(csv_data)

